# Visualize network

The purpose of this notebook is to help visualize the metabolic network via Escher and ensure that the map and model are synchronized with each other.

## Using Escher online:
Brief steps on how to load the RBC metabolic network map using Escher online. 
1. Go to https://escher.github.io/#/ and set both the map and model options to *None*. 
2. Click the *Load map* button to open a blank canvas.
3. Use *(Ctrl+M or Cmd+M)* to load a COBRA model from a `json` file. Alternatively, click on the *Model* tab, then the *Load COBRA model JSON* option to load a new model.
    * Load the model file **"RBC-GEM.json"** from the `/model` directory.
4. Use *(Ctrl+O or Cmd+O)* to load an Escher map from a `json` file. Alternatively, click on the *Map* tab, then the *Load map JSON* option to load a new map.
    * Load the map file **"RBC-GEM.full.map.json"** from the `/map` directory.

## Using Escher via python API (Not recommended currently):
It is currently not recommend to utilize the python API as the current Escher dependencies conflict with recent versions of jupyter. It is therefore up to the user to manage package dependencies to utilize the Python API for Escher. 

The best way to do this is to utilize a seperate virtual environment into prevent dependency conflicts and install Escher. 
1. Install Python 3.9
2. Run the following lines to install packages (order matters!):

    ```
    cd /code # Navigate to code directory where the pyproject.toml file is located.
    pip install markupsafe==2.0.1
    pip install notebook==6.5.6
    pip install escher # Don't worry about any other dependency conflicts
    pip install "." # or ".[all]" for all optional dependencies
    ```

Once dependency conflicts are worked out, this will be updated :) 

## Additional information
See the [documentation for Escher](https://escher.readthedocs.io/en/latest/) for additional details on how to use Escher.


King ZA, Dräger A, Ebrahim A, Sonnenschein N, Lewis NE, Palsson BO. Escher: A Web Application for Building, Sharing, and Embedding Data-Rich Visualizations of Biological Pathways. PLoS Comput Biol. 2015 Aug 27;11(8):e1004321. doi: 10.1371/journal.pcbi.1004321. PMID: 26313928; PMCID: PMC4552468.
## Setup
### Import packages

In [1]:
import datetime
import json
from collections import defaultdict
from pathlib import Path

import matplotlib as mpl
import pandas as pd
import requests
from rbc_gem_utils import (EXTERNAL_PATH, GEM_NAME, GEM_URL, MODEL_PATH,
                           ROOT_PATH, read_cobra_model, read_rbc_model,
                           show_versions)
from rbc_gem_utils.util import ensure_iterable

show_versions()


Package Information
-------------------
rbc-gem-utils 0.0.1

Dependency Information
----------------------
beautifulsoup4                       4.12.3
bio                                   1.6.2
cobra                                0.29.0
depinfo                               2.2.0
kaleido                               0.2.1
matplotlib                            3.8.2
memote                               0.17.0
networkx                              3.2.1
notebook                              7.0.7
openpyxl                              3.1.2
pandas                                2.2.0
pre-commit                            3.6.0
pyvis                                 0.3.2
rbc-gem-utils[database,network,vis] missing
requests                             2.31.0
scipy                                1.12.0
seaborn                              0.13.2

Build Tools Information
-----------------------
pip        23.3.1
setuptools 68.2.2
wheel      0.41.2

Platform Information
-------------------

## Load RBC-GEM model
* Load the XML model to utilize annotations in any data mapping and visualization (guarunteed to have annotations).
* Use the JSON model to check against the model file that gets loaded into Escher.

In [2]:
data_path = Path("data").resolve()
models_path = Path("models").resolve()
figures_path = Path("figures").resolve()
version = "1.2.0"


save_figures = True
transparent = False
imagetype = "svg"
dpi = 600

ftype = "xml"
model = read_cobra_model(f"{models_path}/{GEM_NAME.replace('-', '_')}.{ftype}")
model

Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-21


Name,RBC_GEM
Memory address,147bbea50
Number of metabolites,2157
Number of reactions,3275
Number of genes,820
Number of groups,78
Objective expression,1.0*NaKt - 1.0*NaKt_reverse_db47e
Compartments,"cytosol, extracellular space"


## Load map JSON

In [3]:
save_figures = True

map_name = f"{GEM_NAME.replace('-', '_')}.full.map"
log_level = "OFF"

map_json_filepath = f"{models_path}/{map_name}.json"
map_json_filepath

'/Users/zhaiman/opt/github/RBC-GEM/code/notebooks/studies/rbc1/models/RBC_GEM.full.map.json'

### Format data for viewing on map

In [4]:
try:
    import escher

    escher.rc["never_ask_before_quit"] = True
except ImportError:
    escher = None

In [5]:
reaction_data = {}
metabolite_data = {}
gene_data = {}
reaction_scale = []
metabolite_scale = []

#### Example: categorized by subsystems

In [6]:
df_pathways = pd.read_csv(f"{data_path}/subsystems.tsv", sep="\t", dtype=str)
df_pathways["category"] = df_pathways["category"].replace(
    "Metabolism of other amino acids", "Amino acid metabolism"
)

categories_to_exclude = {"Pseudoreactions", "Model total"}

cmax = 0.8
colors = {
    "Amino acid metabolism": mpl.colors.to_hex(mpl.cm.spring(cmax)),
    "Carbohydrate metabolism": mpl.colors.to_hex(mpl.cm.Greens(cmax)),
    "Lipid metabolism": mpl.colors.to_hex(mpl.cm.Blues(cmax)),
    "Metabolism of cofactors and vitamins": mpl.colors.to_hex(mpl.cm.summer(cmax)),
    "Nucleotide metabolism": mpl.colors.to_hex(mpl.cm.winter(cmax)),
    "Reactive species": mpl.colors.to_hex(mpl.cm.Reds(cmax)),
    "Transport reactions": mpl.colors.to_hex(mpl.cm.Purples(cmax)),
    "Other": mpl.colors.to_hex(mpl.cm.gray_r(cmax)),
}

reaction_scales_mapping = {
    subsystem: {"type": "value", "value": f"{val}", "color": f"{color}"}
    for val, (subsystem, color) in enumerate(colors.items(), start=1)
}


reaction_data = {}
df_cat_subsystems = df_pathways.groupby("category")["name"].agg(lambda x: list(x))
for category, subsystem_list in df_cat_subsystems.items():
    if category in categories_to_exclude:
        continue
    if (
        category not in reaction_scales_mapping
        and category not in categories_to_exclude
    ):
        category = "Other"

    reaction_data.update(
        {
            reaction.id: reaction_scales_mapping[category]["value"]
            for group in model.groups.get_by_any(subsystem_list)
            for reaction in group.members
        }
    )

reaction_scale = list(reaction_scales_mapping.values())
if escher is not None:
    builder = escher.Builder(
        map_json=map_json_filepath,
        # model=model,
        model_json=f"{models_path}/{GEM_NAME.replace('-', '_')}.json",
    )

    for attr, value in dict(
        reaction_data=reaction_data,
        metabolite_data=metabolite_data,
        gene_data=gene_data,
        reaction_scale=reaction_scale,
        metabolite_scale=metabolite_scale,
    ).items():
        if value:
            setattr(builder, attr, value)
    if save_figures:
        builder.save_html(
            f"{figures_path}/{GEM_NAME.replace('-', '_')}_categorized_subsystems.html"
        )

### Export data for web browser

In [7]:
with open(f"{data_path}/reaction_map_data.json", "w") as map_datafile:
    json.dump(reaction_data, map_datafile)

with open(f"{data_path}/metabolite_map_data.json", "w") as map_datafile:
    json.dump(metabolite_data, map_datafile)

### Utilize EscherConverter to convert map to a standard format
To run the Escher converter, uncomment out the final cell
* GitHub Page: https://github.com/draeger-lab/EscherConverter
* Instructions for the Escher converter: https://escher.readthedocs.io/en/stable/escherconverter.html

In [8]:
convert_to_types = [
    # "Escher",
    # "SBGN",
    # "SBML",
]

if convert_to_types:
    for convert_to in convert_to_types:
        converter_path = f"{ROOT_PATH}{EXTERNAL_PATH}/EscherConverter-1.2.1.jar"
        converted_output_filepath = {
            "Escher": f"{data_path}/{map_name}.json",
            "SBGN": f"{data_path}/{map_name}.sbgn",
            "SBML": f"{data_path}/{map_name}.xml",
        }[convert_to]
        !java -jar -Xms8G -Xmx8G -Duser.language=en "$converter_path" --input="$map_json_filepath" --output="$converted_output_filepath" --gui=false --log-level="$log_level"